In [ ]:
import os 
import numpy as np
import cv2
from glob import glob
from PIL import Image
from keras import backend as K
import imutils
import gc
from sklearn.utils import shuffle, Sequence
import random
label_max_len = 37
minibatch_size = 50

In [ ]:
def text_to_labels(text):
    ret = []
    for char in text:
        if char >= 'a' and char <= 'z':
            ret.append(ord(char) - ord('a'))
        elif char == ' ':
            ret.append(26)
    return ret

def labels_to_text(labels):
    # 26 is space, 27 is CTC blank char
    text = ''
    for c in labels:
        if c >= 0 and c < 26:
            text += chr(c + ord('a'))
        elif c == 26:
            text += ' '
    return text

def get_padded_label(label):
    padding = np.ones((label_max_len-len(label))) * 28        # -1  
    return np.concatenate((np.array(label), padding), axis=0)

# def get_unpadded_label(label):
#     padding = np.ones((0)) * -1
#     return np.concatenate((np.array(label), padding), axis=0)

In [ ]:
image_unseen_train = np.load('./data_split/unseen/image_unseen_train.npy')
image_unseen_val = np.load('./data_split/unseen/image_unseen_val.npy')

f_val = open('./data_split/unseen/correct_txt_unseen_val.txt')
txt_unseen_val = []
for line in f_val:
    line = line.strip('\n')
    txt_unseen_val.append(line)
    
f_train = open('./data_split/unseen/correct_txt_unseen_train.txt')
txt_unseen_train = []
for line in f_train:
    line = line.strip('\n')
    txt_unseen_train.append(line)
    
train_label_length = []
train_input_length = []
for i in txt_unseen_train:
    train_label_length.append(len(i))
    train_input_length.append(75)
    
val_label_length = []
val_input_length = []
for i in txt_unseen_val:
    val_label_length.append(len(i))
    val_input_length.append(75)
    
train_label_length = np.array(train_label_length)
train_input_length = np.array(train_input_length)
val_label_length = np.array(val_label_length)
val_input_length = np.array(val_input_length)
txt_unseen_train = np.array(txt_unseen_train)
txt_unseen_val = np.array(txt_unseen_val)

print(image_unseen_train.shape, txt_unseen_train.shape)
print(image_unseen_val.shape, txt_unseen_val.shape)
print(train_label_length.shape, train_input_length.shape)
print(val_label_length.shape, val_input_length.shape)

In [ ]:
def train_batch_gen(data, data_label, input_length, label_length, minibatch_size):  # 定义batch数据生成器
    idx = 0
    while True:
        if idx + minibatch_size>=len(data_label):
            idx=0
#             randnum = random.randint(0,100)
#             random.seed(randnum)
#             random.shuffle(data)
#             random.seed(randnum)
#             random.shuffle(data_label)
#             random.seed(randnum)
#             random.shuffle(input_length)
#             random.seed(randnum)
#             random.shuffle(label_length)

#             data,data_label,input_length,label_length = shuffle(data,data_label,input_length,label_length
#                                                                 ,random_state=13)
        
        start = idx
        idx += minibatch_size
        batch_data = []
        batch_data = data[start:start+minibatch_size]

        batch_label = []
#         batch_label = data_label[start:start+minibatch_size]
        for index in data_label[start:start+minibatch_size]:
            temp = text_to_labels(index)
            temp = get_padded_label(temp)
#             temp = get_unpadded_label(temp)
            batch_label.append(temp)
        batch_label = np.array(batch_label)
        
        batch_input_length = []
        batch_input_length = input_length[start:start+minibatch_size]
        batch_label_length = []
        batch_label_length = label_length[start:start+minibatch_size]
        
        # 畫素資料浮點化以便歸一化
        batch_data = batch_data.astype('float32')
        batch_data /= 255
        
        inputs = {'the_input': batch_data,
                  'the_labels': batch_label,
                  'input_length': batch_input_length,
                  'label_length': batch_label_length}
        outputs = {'ctc': np.zeros([minibatch_size])}
        
        yield (inputs, outputs)

In [ ]:
def val_batch_gen(data, data_label, input_length, label_length, minibatch_size):  # 定义batch数据生成器
    idx = 0
    while True:
        if idx + minibatch_size>=len(data_label):
            idx=0
        
        start = idx
        idx += minibatch_size
        batch_data = []
        batch_data = data[start:start+minibatch_size]
        
        batch_label = []
#         batch_label = data_label[start:start+minibatch_size]
        for index in data_label[start:start+minibatch_size]:
            temp = text_to_labels(index)
            temp = get_padded_label(temp)
#             temp = get_unpadded_label(temp)
            batch_label.append(temp)
        batch_label = np.array(batch_label)
         
        batch_input_length = []
        batch_input_length = input_length[start:start+minibatch_size]
        batch_label_length = []
        batch_label_length = label_length[start:start+minibatch_size]
        
        # 畫素資料浮點化以便歸一化
        batch_data = batch_data.astype('float32')
        batch_data /= 255
        
        inputs = {'the_input': batch_data,
                  'the_labels': batch_label,
                  'input_length': batch_input_length,
                  'label_length': batch_label_length}
        outputs = {'ctc': np.zeros([minibatch_size])}
        
        yield (inputs, outputs)

# model

In [ ]:
from keras import models, layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, SpatialDropout3D
from keras.layers import Convolution3D, MaxPooling3D
from keras.layers.convolutional import Conv3D, ZeroPadding3D
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU
from keras.layers.core import Lambda
from keras.layers import Input
from keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from multiprocessing import set_start_method, Pool
set_start_method('forkserver')
from decoders import Decoder
from callbacks import Statistics, Visualize
from spell import Spell
# from error_rates import ErrorRates

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # From Keras example image_ocr.py:
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    # y_pred = y_pred[:, 2:, :]
    y_pred = y_pred[:, :, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
def CTC(name, args):
    return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)

In [ ]:
class LipNet(object):
    def __init__(self, img_c=3, img_w=100, img_h=50, frames_n=75, absolute_max_string_len=37, output_size=28):
        self.img_c = img_c
        self.img_w = img_w
        self.img_h = img_h
        self.frames_n = frames_n
        self.absolute_max_string_len = absolute_max_string_len
        self.output_size = output_size
        self.build()
    
    def build(self):
        self.input_data = Input(name='the_input', shape=(75,50,100,3), dtype='float32')
        
        self.zero1 = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(self.input_data)
        self.conv1 = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), kernel_initializer='he_normal', name='conv1')(self.zero1)
        self.batc1 = BatchNormalization(name='batc1')(self.conv1)
        self.actv1 = Activation('relu', name='actv1')(self.batc1)
        self.drop1 = SpatialDropout3D(0.5)(self.actv1)
        self.maxp1 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(self.drop1)

        self.zero2 = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(self.maxp1)
        self.conv2 = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv2')(self.zero2)
        self.batc2 = BatchNormalization(name='batc2')(self.conv2)
        self.actv2 = Activation('relu', name='actv2')(self.batc2)
        self.drop2 = SpatialDropout3D(0.5)(self.actv2)
        self.maxp2 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(self.drop2)

        self.zero3 = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(self.maxp2)
        self.conv3 = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv3')(self.zero3)
        self.batc3 = BatchNormalization(name='batc3')(self.conv3)
        self.actv3 = Activation('relu', name='actv3')(self.batc3)
        self.drop3 = SpatialDropout3D(0.5)(self.actv3)
        self.maxp3 = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(self.drop3)

        self.resh1 = TimeDistributed(Flatten())(self.maxp3)

        self.gru_1 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(self.resh1)
        self.gru_2 = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru2'), merge_mode='concat')(self.gru_1)

        # transforms RNN output to character activations:
        self.dense1 = Dense(self.output_size, kernel_initializer='he_normal', name='dense1')(self.gru_2)

        self.y_pred = Activation('softmax', name='softmax')(self.dense1)

        self.labels = Input(name='the_labels', shape=[self.absolute_max_string_len], dtype='float32')
        self.input_length = Input(name='input_length', shape=[1], dtype='int64')
        self.label_length = Input(name='label_length', shape=[1], dtype='int64')

        self.loss_out = CTC('ctc', [self.y_pred, self.labels, self.input_length, self.label_length])

        self.model = Model(inputs=[self.input_data, self.labels, self.input_length, self.label_length], outputs=self.loss_out)
        
    def summary(self):
        Model(inputs=self.input_data, outputs=self.y_pred).summary()

    def predict(self, input_batch):
        return self.test_function([input_batch, 0])[0]  # the first 0 indicates test

    @property
    def test_function(self):
        # captures output of softmax so we can decode the output during visualization
        return K.function([self.input_data, K.learning_phase()], [self.y_pred])

In [ ]:
lipnet = LipNet(img_c=3, img_w=100, img_h=50, frames_n=75, absolute_max_string_len=label_max_len, output_size=28)
lipnet.summary()

In [ ]:
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = adam)

In [ ]:
# 恢复模型结构及权重
# lipnet.model.load_weights('./weight/368-overlap-2.h5')
lipnet.model.load_weights('./weight/unseen-weights178.h5')

In [ ]:
spell = Spell('./grid1.txt')
decoder = Decoder(greedy=False,beam_width=200,postprocessors=[labels_to_text,spell.sentence])
statistics  = Statistics(lipnet, val_batch_gen(image_unseen_val, txt_unseen_val, val_input_length, val_input_length, 50), decoder, 256)
# visualize = Visualize('./unseen_visual', lipnet, val_batch_gen(image_unseen_val, txt_unseen_val, val_input_length, val_input_length, minibatch_size)
#                      , decoder, num_display_sentences=minibatch_size)
# error_rates = ErrorRates(lipnet, val_batch_gen(image_unseen_val, txt_unseen_val, val_input_length, val_input_length, 50), decoder, 256)

In [ ]:
--------------------------------------------------------------------------------------------------------------

In [ ]:
lipnet.model.fit_generator(generator = train_batch_gen(image_unseen_train, txt_unseen_train, train_input_length, train_label_length, minibatch_size),
                        steps_per_epoch = int(len(image_unseen_train)/minibatch_size), epochs = 5,
                        validation_data = val_batch_gen(image_unseen_val, txt_unseen_val, val_input_length, val_input_length, minibatch_size), 
                        validation_steps = int(len(image_unseen_val)/minibatch_size),
                        callbacks = [statistics],
                        initial_epoch = 0,
                        verbose=1,
                        max_q_size=5,
                        workers=0,
#                         pickle_safe=True,
#                         shuffle=True,
                        use_multiprocessing=True)

In [ ]:
--------------------------------------------------------------------------------------------------------------

In [ ]:
# 保存模型结构及权重
lipnet.model.save('./weight/178-unseen-5.h5')

# predict

In [ ]:
# 恢复模型结构及权重
# lipnet.model.load_weights('./weight/368-overlap-6.h5')
lipnet.model.load_weights('./weight/178-unseen-6.h5')

In [ ]:
def predicts(data, input_length):
    batch_data = []
    batch_data = data[495:505]
    
    # 畫素資料浮點化以便歸一化
    batch_data = batch_data.astype('float32')
    batch_data /= 255
    
    batch_input_length = []
    batch_input_length = input_length[495:505]
    
    return (batch_data, batch_input_length)

In [ ]:
pred_data, pred_input_length = predicts(image_unseen_val, val_input_length)

In [ ]:
y_pred = lipnet.predict(pred_data)
# y_predy_pred
result = decoder.decode(y_pred, pred_input_length)
print(result)

In [ ]:
print(txt_unseen_val[495:505])